## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-17-16-51-26 +0000,nyt,Live Updates: Zelensky Will Meet Trump as He P...,https://www.nytimes.com/live/2025/10/17/us/tru...
1,2025-10-17-16-49-00 +0000,wsj,Navy Is Holding Survivors of Latest U.S. Strik...,https://www.wsj.com/world/americas/drug-boat-s...
2,2025-10-17-16-48-28 +0000,nyt,"Susan Stamberg, Longtime Mainstay of NPR, Dies...",https://www.nytimes.com/2025/10/16/business/me...
3,2025-10-17-16-46-08 +0000,bbc,Fears over US banks cause stock market jitters,https://www.bbc.com/news/articles/c4gjz82wx4wo...
4,2025-10-17-16-44-41 +0000,cbc,International deal to cut shipping emissions f...,https://www.cbc.ca/news/climate/imo-shipping-e...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,60
8,new,18
298,china,16
6,he,13
16,against,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
306,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...,138
0,2025-10-17-16-51-26 +0000,nyt,Live Updates: Zelensky Will Meet Trump as He P...,https://www.nytimes.com/live/2025/10/17/us/tru...,131
362,2025-10-16-17-33-15 +0000,nypost,Trump says he will meet with Putin in Budapest...,https://nypost.com/2025/10/16/us-news/trump-sa...,125
311,2025-10-16-20-34-59 +0000,nypost,Ex-Trump national security adviser John Bolton...,https://nypost.com/2025/10/16/us-news/ex-trump...,123
266,2025-10-16-23-01-00 +0000,wsj,Trump Says He Will Meet With Putin in Budapest...,https://www.wsj.com/world/russia/trump-and-put...,122


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
306,138,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...
210,73,2025-10-17-03-46-08 +0000,nypost,"Zohran Mamdani dodges attacks, straight answer...",https://nypost.com/2025/10/16/us-news/mamdani-...
311,63,2025-10-16-20-34-59 +0000,nypost,Ex-Trump national security adviser John Bolton...,https://nypost.com/2025/10/16/us-news/ex-trump...
0,50,2025-10-17-16-51-26 +0000,nyt,Live Updates: Zelensky Will Meet Trump as He P...,https://www.nytimes.com/live/2025/10/17/us/tru...
121,42,2025-10-17-11-31-08 +0000,bbc,MPs demand chief prosecutor explain China spy ...,https://www.bbc.com/news/articles/cvgd5jgzdweo...
81,39,2025-10-17-14-05-00 +0000,wsj,Funding for federal agencies lapsed beginning ...,https://www.wsj.com/politics/policy/federal-go...
362,38,2025-10-16-17-33-15 +0000,nypost,Trump says he will meet with Putin in Budapest...,https://nypost.com/2025/10/16/us-news/trump-sa...
55,38,2025-10-17-15-18-45 +0000,nyt,"Baek Sehee, South Korean Author of ‘I Want to ...",https://www.nytimes.com/2025/10/17/arts/baek-s...
76,37,2025-10-17-14-23-12 +0000,nypost,Tattooed salon owner allegedly threatened to k...,https://nypost.com/2025/10/17/us-news/bat-cave...
134,37,2025-10-17-11-00-01 +0000,startribune,"Hit by an e-bike, she can no longer taste or s...",https://www.startribune.com/minnesota-ebike-sp...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
